In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import datetime
import tensorflow as tf
import keras
import time

Using TensorFlow backend.


In [2]:
class GraphGRU(keras.layers.Layer):
    def __init__(self, neurons=32, nodesN=10):
        super(GraphGRU, self).__init__()
        wrgg_init = tf.random_normal_initializer()
        self.wrgg = tf.Variable(
            initial_value=wrgg_init(shape=(neurons,nodesN,nodesN), dtype="float32"),
            trainable=True,
        )
        wrr_init = tf.random_normal_initializer()
        self.wrr = tf.Variable(
            initial_value=wrr_init(shape=(neurons,neurons), dtype="float32"),
            trainable=True,
        )
        br_init = tf.zeros_initializer()
        self.br = tf.Variable(
            initial_value=br_init(shape=(neurons,), dtype="float32"), trainable=True
        )
        wzgg_init = tf.random_normal_initializer()
        self.wzgg = tf.Variable(
            initial_value=wzgg_init(shape=(neurons,nodesN,nodesN), dtype="float32"),
            trainable=True,
        )
        wzz_init = tf.random_normal_initializer()
        self.wzz = tf.Variable(
            initial_value=wzz_init(shape=(neurons,neurons), dtype="float32"),
            trainable=True,
        )
        bz_init = tf.zeros_initializer()
        self.bz = tf.Variable(
            initial_value=bz_init(shape=(neurons,), dtype="float32"), trainable=True
        )
        whgg_init = tf.random_normal_initializer()
        self.whgg = tf.Variable(
            initial_value=whgg_init(shape=(neurons,nodesN,nodesN), dtype="float32"),
            trainable=True,
        )
        whh_init = tf.random_normal_initializer()
        self.whh = tf.Variable(
            initial_value=whh_init(shape=(neurons,neurons), dtype="float32"),
            trainable=True,
        )
        bh_init = tf.zeros_initializer()
        self.bh = tf.Variable(
            initial_value=bh_init(shape=(neurons,), dtype="float32"), trainable=True
        )

    def call(self,inputs):
# inputs is a list of tensors. It seems that I can't have two arguments
# to a call to a layer (maybe? in any case get an error when I try). So
# one argument as a list of tensors [memory_in,graph] seems to workaround.
        memory_in=inputs[0]
        graph=inputs[1]
        print('graph.shape',graph.shape)
        print('memory_in.shape',memory_in.shape)
        memory_in=tf.reshape(memory_in,[memory_in[0],1])
        Rr=tf.reduce_sum(tf.math.multiply(self.wrgg[:],graph),[1,2])
        Rh=tf.linalg.matmul(self.wrr,memory_in)
        R=tf.math.sigmoid(Rr+Rh+self.br)
        
        Zz=tf.reduce_sum(tf.math.multiply(self.wzgg[:],graph),[1,2])
        Zh=tf.linalg.matmul(self.wzz,memory_in)
        Z=tf.math.sigmoid(Zz+Zh+self.bz)
        
        hh1=tf.math.multiply(R,memory_in)
        H01=tf.reduce_sum(tf.math.multiply(self.whgg[:],graph),[1,2])
        H02=tf.linalg.matmul(self.whh,hh1)
        H0=tf.math.tanh(H01+H02+self.bh)
        
        H=tf.math.multiply(Z,memory_in)+tf.math.multiply((1-Z),H0)
        
        return H

In [3]:
x=tf.constant(([[[1,2,3],[4,5,6],[7,8,9]],[[0.2,0.4,0.6],[-1,-2,-3],[3,4,5]]]))
y=tf.constant(([[2.2,3,4],[9,8,7],[1,2,3]]))
z=tf.math.multiply(x[:],y)
v=y+y
w=tf.reduce_sum(z,0)
with tf.Session() as sess:  print(x.eval(),"\n y \n",y.eval(),"\n z \n",z.eval(),"\n w \n",w.eval(),"v",v.eval())
layer=GraphGRU(25,15)
hm1=tf.random.normal([25])
edges=tf.random.normal([15,15])
nodes=tf.random.normal([15])
inputs=[hm1,edges,nodes]
x1=layer(inputs)
print(x1)

[[[ 1.   2.   3. ]
  [ 4.   5.   6. ]
  [ 7.   8.   9. ]]

 [[ 0.2  0.4  0.6]
  [-1.  -2.  -3. ]
  [ 3.   4.   5. ]]] 
 y 
 [[2.2 3.  4. ]
 [9.  8.  7. ]
 [1.  2.  3. ]] 
 z 
 [[[  2.2          6.          12.        ]
  [ 36.          40.          42.        ]
  [  7.          16.          27.        ]]

 [[  0.44000003   1.2          2.4       ]
  [ -9.         -16.         -21.        ]
  [  3.           8.          15.        ]]] 
 w 
 [[ 2.64  7.2  14.4 ]
 [27.   24.   21.  ]
 [10.   24.   42.  ]] v [[ 4.4  6.   8. ]
 [18.  16.  14. ]
 [ 2.   4.   6. ]]

graph.shape (15, 15)
memory_in.shape (25,)
Tensor("graph_gru_1/add_6:0", shape=(25, 25), dtype=float32)


In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs[0], self.w) + self.b

In [3]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)


In [ ]:
# train the model
def build_model(train_x,train_y,test_x,test_y,timesteps,ndays_in,features,epochs,neurons):
    print('train_x.shape ',train_x.shape,'train_y.shape ',train_y.shape)
    verbose, batch_size = 1, 80
# reshape train_x and train_y into [samples, timesteps, features] - ***this is trivial***
    train_x=train_x.reshape((train_x.shape[0],ndays_in,features)) # train_x.reshape((train_x.shape[0],train_x.shape[1],1))
    train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
    test_x=test_x.reshape((test_x.shape[0],ndays_in,features)) # test_x.reshape((train_x.shape[0],train_x.shape[1],1))
    test_y = test_y.reshape((test_y.shape[0], test_y.shape[1], 1))
    n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
    print('train_y.shape ',train_y.shape,' train_x.shape ',train_x.shape)
        
# define model
#    neurons=400
    denseL=200
    model = Sequential()
    model.add(LSTM(neurons, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_outputs)) # this makes n_outputs (rather than 1, as in Dense(1))
    model.add(LSTM(neurons, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(denseL, activation='relu')))
    model.add(TimeDistributed(Dense(1)))
#    opt=keras.optimizers.Adam(lr=0.005)
    model.compile(loss='mse', optimizer='adam')
# save inital weioghts
    model.save_weights('modelW.h5')
# fit network
#    history=model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose, \
#                     callbacks=[MyCustomCallback()])


    history=model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size, verbose=verbose, \
                     validation_data=(test_x,test_y))
#        ilayer=-1
#        for layer in model.layers:
#            weights=layer.get_weights()
#            biases=layer.get_weights()[1]
#            print(' layer \n',layer,' weights.shape ',weights.shape)

    return model, history


In [5]:
layer = MyDenseLayer(10)
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.
print([var.name for var in layer.trainable_variables])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
['my_dense_layer_1/kernel:0']


In [3]:
x = tf.ones((2, 2))
x2 = tf.ones((3,3))
x3=[x,x2]
linear_layer = Linear(4, 2)
y = linear_layer(x3)
print(y)


Tensor("linear_1/add:0", shape=(2, 4), dtype=float32)
